This notebook compares the accuracy of multiple data driven ROM methods on the full assembly

In [17]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
import numpy as np
from sklearn.decomposition import TruncatedSVD
from scipy.sparse import csr_matrix
from sklearn.utils.extmath import randomized_svd
 
import os
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.animation import FuncAnimation
from dotenv import dotenv_values 
from moviepy.editor import VideoClip
from moviepy.video.io.bindings import mplfig_to_npimage
import scipy.io
from scipy.signal import cont2discrete
import pyvista as pv
import h5py
import mat73
import dask.array as da
from dask.distributed import Client, LocalCluster




import opinf


config = dotenv_values(".env")
from util import *

In [19]:
# Setup Dask client
cluster = LocalCluster() # Launches a scheduler and workers locally
client = Client(cluster) # Connect to distributed cluster and override default
# client = Client('tcp://127.0.0.1:8786')
client.cluster

LocalCluster(8820b9f6, 'tcp://127.0.0.1:41075', workers=6, threads=24, memory=62.50 GiB)

In [32]:
# Set up hyperparameters for all ROMs
n_timesteps = 2000 # Number of timesteps to pull from each episode
dt = 0.001
n_train = 2 # Number of training episodes
n_test = 0 # Number of testing episodes
n_ROM = 22 # Dimension of ROM state space

# Read Data

In [33]:
# Setup filepaths for reading data 
filepath = config["currentDirectory"] + "data/archivedDataSets/FullAssembly_Constrained_FullSetForICRA_wrongMatParams/"



In [34]:
# Read in all training data 
data = h5py.File(filepath+ "fullDataSet.hdf5", 'r')
print("gothere")
X_fom = da.from_array(data["stateData"][:,:,0:n_train+n_test], chunks=(4096, 4096,1))
U_fom = da.from_array(data["inputData"][:,:,0:n_train+n_test], chunks=(4096, 4096,1))
Y_fom = da.from_array(data["reducedCenterlineData"][:,:,0:n_train+n_test], chunks=(4096, 4096,1))
x0 = data["stateData"][:,0,0]
y0 = data["reducedCenterlineData"][:,0,0]


# del data # free up memory
print("gothere2")
n = X_fom.shape[0]
l = U_fom.shape[0]
m = Y_fom.shape[0]
print("gothere3")
# del data # free up memory
# Collect initial condition offsets for each episode and center each episode
X0 = da.zeros((n,n_train+n_test))
Y0 = da.zeros((m,n_train+n_test))
for i in range(n_train+n_test):
    X0[:,i] = X_fom[:,0,i]
    Y0[:,i] = Y_fom[:,0,i]
    X_fom[:,:,i] = X_fom[:,:,i] - X0[:,i].reshape(-1,1)
    Y_fom[:,:,i] = Y_fom[:,:,i] - Y0[:,i].reshape(-1,1)


gothere
gothere2
gothere3


In [ ]:
data["reducedCenterlineData"][:,0,0]


In [ ]:
x0.shape

In [ ]:
# Get length of robot from min and max of x coords
xzCoords = Y0[:,0].reshape(-1,2)
x_min = np.min(xzCoords[:,0])
x_max = np.max(xzCoords[:,0])
length = x_max - x_min
print("Length of robot: ", length.compute())
# Get z min and max
z_min = np.min(xzCoords[:,1])
z_max = np.max(xzCoords[:,1])

In [ ]:
# Print shapes of data
print("Shape of X_fom: ", X_fom.shape)
print("Shape of U_fom: ", U_fom.shape)
print("Shape of Y_fom: ", Y_fom.shape)

# Learn System Matrices for Each Method

## SVD to find correct size of bases

In [ ]:
# Compute SVD of state training set to find size of reduced basis
# Unroll the state training set into a matrix
X_fom_train_da = da.hstack([X_fom[:,:,i] for i in range(n_train)])
# Get shape of state training set
print(X_fom_train_da.shape)
X_fom_train_da

In [ ]:
# compute singular values of unrolled state training set
U_fom_train_da, S_fom_train_da, V_fom_train_da = da.linalg.svd_compressed(X_fom_train_da, 1000)  
# Compute normalized singular values
S_fom_train_norm = S_fom_train_da / da.sum(S_fom_train_da)


In [ ]:
S_fom_train_norm

In [ ]:
S_fom_train_norm = S_fom_train_norm.compute()


In [ ]:
# Plot normalized singular values and the cumulative sum of normalized singular values
plt.figure(figsize=(10,5))
plt.subplot(1,2,1)
plt.plot(S_fom_train_norm)
plt.title('Normalized singular values')
plt.xlabel('Singular value index')
plt.ylabel('Normalized singular value')
plt.axvline(x=22, color='r', linestyle='--')

plt.subplot(1,2,2)
plt.plot(np.cumsum(S_fom_train_norm))
plt.title('Cumulative sum of normalized singular values')
plt.xlabel('Singular value index')
plt.ylabel('Cumulative sum of normalized singular values')
# Plot a vertical line at 22
plt.axvline(x=22, color='r', linestyle='--')

print(np.cumsum(S_fom_train_norm))


In [ ]:
n_ROM = 22 # Dimensionality of state in ROM

## OKID + ERA 

### Setup Data matrices  

In [ ]:
# Grab input-output data from single episode 
U_era = U_fom[:,:,0]
Y_era = Y_fom[:,:,0]
# # Center output data around initial conditions
# Y_offset_era = Y_era[:,0].reshape(-1,1)
# Y_era = Y_era - Y_offset_era



### OKID Algorithm 

In [ ]:
YY = Y_era
UU_size = (U_era.shape[0]*U_era.shape[1],U_era.shape[1])
UU = da.zeros(UU_size)

for i in range(U_era.shape[1]):
    startRow = i*U_era.shape[0]
    endRow = (i+1)*U_era.shape[0]
    UU[startRow:endRow,i:] = U_era[:,0:(U_era.shape[1]-i)]

# compute pseudoinverse of input matrix using SVD
U_UU, S_UU, Vh_UU = da.linalg.svd(UU)
UU_pinv = da.matmul(da.transpose(Vh_UU),da.matmul(da.diag(1/S_UU),da.transpose(U_UU)))
# UU_pinv = da.linalg.pinv(UU)
# Solve for impulse response  
IR = da.matmul(YY,UU_pinv)

# IR = da.linalg.lstsq(da.transpose(YY),da.transpose(UU)) # solves for transpose of impulse response

# IR = da.transpose(IR) # transpose to get impulse response


In [ ]:
# Compute things to limit growth of computation graph
IR = IR.compute()
print(IR.shape)
# IR = da.from_array(IR)
print("done")

### Eigensystem Realization Algorithm

In [ ]:
# Form Hankel matrix for ERA 
N = int(n_timesteps/2-1)
H_size = (m*N,l*N)

H = np.zeros(H_size)
Hprime = np.zeros(H_size)

for i in range(N):
    startRow = m*i 
    endRow = m*(i+1)
    H[startRow:endRow,:]=IR[:,l*(1+i):l*(1+i+N)]
    Hprime[startRow:endRow,:]=IR[:,l*(2+i):l*(2+i+N)]


In [ ]:
IR.shape

In [ ]:
H = da.from_array(H, chunks=(2048,H.shape[1]))
Hprime = da.from_array(Hprime, chunks=(2048,Hprime.shape[1]))
IR = da.from_array(IR, chunks=(IR.shape[0],4096))

In [ ]:
# Get SVD of Hankel matrix
U, S, Vh = da.linalg.svd_compressed(H, 1000)
# Vh = V.conj().T

In [ ]:
# Print shapes of matrices
print("Shape of U: ", U.shape)
print("Shape of S: ", S.shape)
print("Shape of Vh: ", Vh.shape)

In [ ]:
# Truncate SVD
U = U[:,:n_ROM]
S = da.diag(S[:n_ROM])
Vh = Vh[:n_ROM,:]

In [ ]:
# Print shapes of matrices
print("Shape of U: ", U.shape)
print("Shape of S: ", S.shape)
print("Shape of Vh: ", Vh.shape)

In [ ]:
# Compute trunacted SVD values
U, S, Vh = da.compute(U, S, Vh)
# S = S.compute()
# Vh = Vh.compute()

In [ ]:
# Print sizes of SVD matrices
print("SVD matrices:")
print("U: ", U.shape)
print("S: ", S.shape)
print("V: ", Vh.shape)

In [ ]:
# Convert np arrays back into dask arrays
U = da.from_array(U)
Vh = da.from_array(Vh)
S = da.from_array(S)

In [ ]:
# Solve for system matrices
Sigma_tilde_is = da.linalg.inv(da.sqrt(S)) # inverse squareroot of sigma matrix
Sigma_tilde_sqrt = da.sqrt(S) # squareroot of sigma matrix


Em = da.concatenate([da.eye(l),da.zeros((l*(N-1),l))],axis=0)
Ep = da.concatenate([da.eye(m),da.zeros((m*(N-1),m))],axis=0)

A_era = da.matmul(da.matmul(da.matmul(da.matmul(Sigma_tilde_is,da.transpose(U)),Hprime),da.transpose(Vh)),Sigma_tilde_is)
B_era = da.matmul(da.matmul(Sigma_tilde_sqrt,Vh),Em)
C_era = da.matmul(da.transpose(Ep),da.matmul(U,Sigma_tilde_sqrt))
D_era = IR[:,:l]


In [ ]:
# A_era = A_era.compute()
# B_era = B_era.compute()
# C_era = C_era.compute()
# D_era = D_era.compute()

A_era, B_era, C_era, D_era = da.compute(A_era, B_era, C_era, D_era)

In [ ]:
# Save system matrices, projection mappings, and initial offset into file for later use
np.savez(filepath+f"eraSystemMatrices_{n_ROM}dim_{n_train}train_{n_test}test.npz", A_era=A_era, B_era=B_era, C_era=C_era, D_era=D_era, x0=x0, y0=y0)
# save as .mat file
scipy.io.savemat(filepath+f"eraSystemMatrices_{n_ROM}dim_{n_train}train_{n_test}test.mat", mdict={'A_era': A_era, 'B_era': B_era, 'C_era': C_era, 'D_era': D_era, 'x0': x0, 'y0': y0})

In [ ]:
# clean up memory except for system matrices
del U_era, Y_era, YY, UU, IR, H, Hprime, U, S, Vh, Sigma_tilde_is, Sigma_tilde_sqrt, Em, Ep


## DMDc

### Setup Data Matrices

In [35]:
# Initialize data matrices for training and testing
X_train = da.zeros((n,(n_timesteps-1)*n_train), chunks=(128,(n_timesteps-1)*n_train))
Xprime_train = da.zeros((n,(n_timesteps-1)*n_train), chunks=(128,(n_timesteps-1)*n_train))
Upsilon_train = da.zeros((l,(n_timesteps-1)*n_train), chunks=(l,512))
Y_train = da.zeros((m,(n_timesteps-1)*n_train), chunks=(m,128))
# Load in data from training episodes
for i in range(n_train):
    X_train[:,i*(n_timesteps-1):(i+1)*(n_timesteps-1)] = X_fom[:,:-1,i]
    Xprime_train[:,i*(n_timesteps-1):(i+1)*(n_timesteps-1)] = X_fom[:,1:,i]
    Upsilon_train[:,i*(n_timesteps-1):(i+1)*(n_timesteps-1)] = U_fom[:,:-1,i]
    Y_train[:,i*(n_timesteps-1):(i+1)*(n_timesteps-1)] = Y_fom[:,:-1,i]


In [36]:
# Form snapshot matrices for DMDc
Omega = da.concatenate([X_train,Upsilon_train],axis=0)
# Omega = Omega.rechunk((Omega.shape[0],1))
# Omega


### Solve for Discrete Time System Matrices

In [37]:
# Decompose snapshot matrix using truncated SVD
p_dmd = 1024
r_dmd = n_ROM

U_tilde,Sigma_tilde,Vh_tilde = da.linalg.svd_compressed(Omega, p_dmd)
# Vh_tilde = da.conj(da.transpose(V_tilde))
Sigma_tilde = da.diag(Sigma_tilde)
U_hat, Sigma_hat, V_hat = da.linalg.svd_compressed(Xprime_train, r_dmd)
Sigma_hat = da.diag(Sigma_hat)



In [38]:
# Print shapes of svd matrices
print("U_tilde shape: ", U_tilde.shape)
print("Sigma_tilde shape: ", Sigma_tilde.shape)
print("Vh_tilde shape: ", Vh_tilde.shape)

U_tilde shape:  (231342, 1024)
Sigma_tilde shape:  (1024, 1024)
Vh_tilde shape:  (1024, 3998)


In [39]:
# Compute system state evolution matrices
U_tilde_1 = U_tilde[0:n,:]
U_tilde_2 = U_tilde[n:,:]
A_dmdc = (U_hat.conj().T)@Xprime_train@(Vh_tilde.conj().T)@(da.linalg.inv(Sigma_tilde))@(U_tilde_1.conj().T)@U_hat
B_dmdc = (U_hat.conj().T)@Xprime_train@(Vh_tilde.conj().T)@(da.linalg.inv(Sigma_tilde))@(U_tilde_2.conj().T)

In [ ]:
# U_tilde_1.compute()

In [40]:
# Solve for system output matrix from full order system
ys = Y_train[:,0:10]

# compute approximate pseudoinverse of data matrix using SVD
# X_train = X_train.rechunk((2000,X_train.shape[1])) # Rechunk along one dimension
U_X_train, S_X_train, Vh_X_train = da.linalg.svd(X_train[:,0:10])
X_PI = da.matmul(da.transpose(Vh_X_train),da.matmul(da.diag(S_X_train),da.transpose(U_X_train)))


# # X_PI = np.linalg.pinv(X_train[:,0:10])
C_dmdc = ys@X_PI
C_dmdc = C_dmdc@U_hat



In [30]:
# Print shape of svd matrices
print("Shape of U_X_train: ", U_X_train.shape)
print("Shape of S_X_train: ", S_X_train.shape)
print("Shape of Vh_X_train: ", Vh_X_train.shape)

Shape of U_X_train:  (231336, 10)
Shape of S_X_train:  (10,)
Shape of Vh_X_train:  (10, 10)


In [41]:
# Execute computations for dmdc system matrices
print("Computing System Matrices")
A_dmdc, B_dmdc, C_dmdc,U_hat = da.compute(A_dmdc, B_dmdc, C_dmdc,U_hat)
# A_dmdc = A_dmdc.compute()
# print("Computing B_dmdc")
# B_dmdc = B_dmdc.compute()
# print("Computing C_dmdc")
# C_dmdc = C_dmdc.compute()
# print("Done")



Computing System Matrices


2023-11-29 12:01:40,497 - distributed.protocol.core - CRITICAL - Failed to Serialize
Traceback (most recent call last):
  File "/home/brdl/.local/lib/python3.8/site-packages/distributed/protocol/core.py", line 109, in dumps
    frames[0] = msgpack.dumps(msg, default=_encode_default, use_bin_type=True)
  File "/home/brdl/.local/lib/python3.8/site-packages/msgpack/__init__.py", line 38, in packb
    return Packer(**kwargs).pack(o)
  File "msgpack/_packer.pyx", line 294, in msgpack._cmsgpack.Packer.pack
  File "msgpack/_packer.pyx", line 300, in msgpack._cmsgpack.Packer.pack
  File "msgpack/_packer.pyx", line 297, in msgpack._cmsgpack.Packer.pack
  File "msgpack/_packer.pyx", line 264, in msgpack._cmsgpack.Packer._pack
  File "msgpack/_packer.pyx", line 231, in msgpack._cmsgpack.Packer._pack
  File "msgpack/_packer.pyx", line 264, in msgpack._cmsgpack.Packer._pack
  File "msgpack/_packer.pyx", line 202, in msgpack._cmsgpack.Packer._pack
ValueError: bytes object is too large
2023-11-29 12:

CancelledError: ('mul-b37b33a9591eb9a315259809b4df2d1f', 909, 0)

In [ ]:
basis_dmdc = U_hat[:,0:n_ROM]


In [ ]:
# Compute U_hat for saving
# U_hat = U_hat.compute()

In [ ]:
# Save system matrices, projection mappings, and initial offset into file for later use
np.savez(filepath+f"dmdcSystemMatrices_{n_ROM}dim_{n_train}train_{n_test}test.npz", A_dmdc=A_dmdc, B_dmdc=B_dmdc, C_dmdc=C_dmdc, basis_dmdc = U_hat, x0=x0, y0=y0)
# save as .mat file
scipy.io.savemat(filepath+f"dmdcSystemMatrices_{n_ROM}dim_{n_train}train_{n_test}test.mat", mdict={'A_dmdc': A_dmdc, 'B_dmdc': B_dmdc, 'C_dmdc': C_dmdc, 'basis_dmdc': U_hat, 'x0': x0, 'y0': y0})

In [ ]:
# Clean up memory except for system matrices
del  Xprime_train, Upsilon_train, Omega, U_tilde, Sigma_tilde, Vh_tilde, Sigma_hat, V_hat, U_tilde_1, U_tilde_2, X_PI, ys

## LOpInf

In [ ]:
# Read in continuous time operators from file
mat = scipy.io.loadmat(filepath+'lopinf_rom_11_operators.mat')
Bhat_lopinf = mat['Bhat']
Chat_lopinf = mat['Chat']
Ehat_lopinf = mat['Cphat']
Khat_lopinf = mat['Khat']
Mhat_lopinf = mat['Mhat']
basis_lopinf = np.hstack((mat['v'],np.zeros((mat['v'].shape[0],mat['v'].shape[1]))))
# Form continuous time linear state space matrices
A_lopinf_ct = np.block([
    [np.zeros((n_ROM//2,n_ROM//2)),np.eye(n_ROM//2)],
    [-Khat_lopinf, -Chat_lopinf]
])
B_lopinf_ct = np.block([
    [np.zeros((n_ROM//2,l))],
    [Bhat_lopinf]
])
C_lopinf_ct = np.block([
    [Ehat_lopinf, np.zeros((m,n_ROM//2))]
])
D_lopinf_ct = np.zeros((m,l))
# Convert continuous time operators to discrete time operators
A_lopinf_dt = scipy.linalg.expm(A_lopinf_ct*dt)
B_lopinf_dt = (A_lopinf_dt-np.eye(n_ROM))@np.linalg.inv(A_lopinf_ct)@B_lopinf_ct
C_lopinf_dt = C_lopinf_ct
D_lopinf_dt = D_lopinf_ct

# A_lopinf, B_lopinf, C_lopinf, D_lopinf, dt_lopinf = cont2discrete((A_lopinf_ct, B_lopinf_ct, C_lopinf_ct,D_lopinf_ct), dt, method='zoh')

In [ ]:
# Print sizes of A matrices
# print('A_dmdc: ', A_dmdc.shape)
# print('A_opInf: ', A_opInf.shape)
# print('A_lopinf: ', A_lopinf.shape)
# Check if lopinf matrices are identical betweeen continuous and discrete time 
# print(np.equal(C_lopinf_dt,C_lopinf).all())
# print(np.equal(D_lopinf_dt,D_lopinf).all())
# print(np.equal(A_lopinf_dt,A_lopinf).all())
# print(np.equal(B_lopinf_dt,B_lopinf).all())



In [ ]:
# Save system matrices, projection mappings, and initial offset into file for later use
np.savez(filepath+f"lopinfSystemMatrices_{n_ROM}dim_{n_train}train_{n_test}test.npz", A_lopinf=A_lopinf_dt,B_lopinf=B_lopinf_dt,C_lopinf=C_lopinf_dt,D_lopinf = D_lopinf_dt,basis_lopinf = mat["v"], x0=x0, y0=y0)
# save as .mat file
scipy.io.savemat(filepath+f"dmdcSystemMatrices_{n_ROM}dim_{n_train}train_{n_test}test.mat", mdict={'A_lopinf': A_lopinf_dt, 'B_lopinf': B_lopinf_dt, 'C_lopinf': C_lopinf_dt, 'D_lopinf': D_lopinf_dt, 'basis_lopinf':mat["v"], 'x0': x0, 'y0': y0})

# Save System Matrices for Control Experiments


In [ ]:
# Save system matrices, projection mappings, and initial offset into file for later use
np.savez(filepath+f"romSystemMatrices_{n_ROM}dim_{n_train}train_{n_test}test.npz", A_era=A_era, B_era=B_era, C_era=C_era, D_era=D_era, A_dmdc=A_dmdc, B_dmdc=B_dmdc, C_dmdc=C_dmdc, basis_dmdc = U_hat, A_lopinf=A_lopinf_dt,B_lopinf=B_lopinf_dt,C_lopinf=C_lopinf_dt,D_lopinf = D_lopinf_dt,basis_lopinf = mat["v"], x0=x0, y0=y0)
# save as .mat file
scipy.io.savemat(filepath+f"romSystemMatrices_{n_ROM}dim_{n_train}train_{n_test}test.mat", mdict={'A_era': A_era, 'B_era': B_era, 'C_era': C_era, 'D_era': D_era, 'A_dmdc': A_dmdc, 'B_dmdc': B_dmdc, 'C_dmdc': C_dmdc, 'basis_dmdc': U_hat, 'A_lopinf': A_lopinf_dt, 'B_lopinf': B_lopinf_dt, 'C_lopinf': C_lopinf_dt, 'D_lopinf': D_lopinf_dt, 'basis_lopinf':mat["v"], 'x0': x0, 'y0': y0})

# Verification Experiments

## Simulate System Responses from Training and Test Episodes

### OKID/ERA

In [ ]:
# Initialize data matrices for training and testing responses
X_hat_rom_era = np.zeros((n_ROM,n_timesteps,n_train+n_test))
Y_hat_rom_era = np.zeros((m,n_timesteps,n_train+n_test))
U_fom = U_fom.compute()
# Set initial conditions for each episode
for i in range(n_train+n_test):
    # Set initial states
    X_hat_rom_era[:,[0],i] = np.zeros((n_ROM,1)) #X_fom[:,[0],i]
    # Compute initial outputs for each episode
    Y_hat_rom_era[:,[0],i] = C_era@X_hat_rom_era[:,[0],i] + D_era@U_fom[:,[0],i]


In [ ]:
# Simulate ERA ROM for all episodes
for i in range(n_train+n_test):
    # Simulate ERA ROM
    for j in range(n_timesteps-1):
        X_hat_rom_era[:,[j+1],i] = A_era@X_hat_rom_era[:,[j],i] + B_era@U_fom[:,[j],i]
        Y_hat_rom_era[:,[j+1],i] = C_era@X_hat_rom_era[:,[j+1],i] + D_era@U_fom[:,[j+1],i]


In [ ]:
# Uncenter output data with initial conditions for each episode
for i in range(n_train+n_test):
    Y_hat_rom_era[:,:,i] = Y_hat_rom_era[:,:,i] + Y0[:,i].reshape(m,1)


In [ ]:
# Clear some memory
del X_hat_rom_era

### DMDc

In [ ]:
# Initialize data matrices for training and testing responses
X_hat_rom_dmdc = np.zeros((n_ROM,n_timesteps,n_train+n_test))
Y_hat_rom_dmdc = np.zeros((m,n_timesteps,n_train+n_test))
# Set initial conditions for each episode
for i in range(n_train+n_test):
    # Set initial states
    X_hat_rom_dmdc[:,[0],i] = np.zeros((n_ROM,1)) #X_fom[:,[0],i]
    # X_hat_fom_dmdc[:,[0],i] = X_fom[:,[0],i]
    # Compute initial outputs for each episode
    Y_hat_rom_dmdc[:,[0],i] = C_dmdc@X_hat_rom_dmdc[:,[0],i]

In [ ]:
# Simulate DMDc ROM for all episodes
for i in range(n_train+n_test):
    # Simulate DMDc ROM
    for j in range(n_timesteps-1):
        X_hat_rom_dmdc[:,[j+1],i] = A_dmdc@X_hat_rom_dmdc[:,[j],i] + B_dmdc@U_fom[:,[j],i]
        Y_hat_rom_dmdc[:,[j+1],i] = C_dmdc@X_hat_rom_dmdc[:,[j+1],i]

In [ ]:
# Uncenter FOM state and output data with initial conditions for each episode
for i in range(n_train+n_test):
    Y_hat_rom_dmdc[:,:,i] = Y_hat_rom_dmdc[:,:,i] + Y0[:,i].reshape(m,1)

In [ ]:
# Clear some memory


### LOpInf

In [ ]:
# Read in data from LOpInf Trials

# Read in data from LOpInf Trials hdf5 file
data = h5py.File(filepath+ "lopinf_rom_11_training.hdf5", 'r')
print("gothere")
Y_hat_rom_lopinf = da.from_array(data["outputData_train_approx"], chunks=(4096, 4096,1))
X_hat_rom_lopinf = da.from_array(data["stateData_train_approx"], chunks=(4096, 4096,1))




# mat = mat73.loadmat(filepath+'lopinf_rom_r_6.mat')
# X_hat_rom_lopinf = mat['stateData_approx']
# Y_hat_rom_lopinf = mat['outputData_approx']


In [ ]:
# Also generate outputs from LOpInf discrete time model
# Initialize data matrices for training and testing responses
X_hat_rom_lopinf_dt = np.zeros((n_ROM,n_timesteps,n_train+n_test))
X_hat_fom_lopinf_dt = np.zeros((n,n_timesteps,n_train+n_test))
Y_hat_rom_lopinf_dt = np.zeros((m,n_timesteps,n_train+n_test))
# Set initial conditions for each episode
for i in range(n_train+n_test):
    # Set initial states
    X_hat_rom_lopinf_dt[:,[0],i] = np.zeros((n_ROM,1)) #X_fom[:,[0],i]
    # X_hat_fom_lopinf_dt[:,[0],i] = X_fom[:,[0],i]
    # Compute initial outputs for each episode
    Y_hat_rom_lopinf_dt[:,[0],i] = C_lopinf_dt@X_hat_rom_lopinf_dt[:,[0],i]


In [ ]:
# Simulate LOpInf discrete time ROM for all episodes
for i in range(n_train+n_test):
    # Simulate LOpInf discrete time ROM
    for j in range(n_timesteps-1):
        X_hat_rom_lopinf_dt[:,[j+1],i] = A_lopinf_dt@X_hat_rom_lopinf_dt[:,[j],i] + B_lopinf_dt@U_fom[:,[j],i]
        Y_hat_rom_lopinf_dt[:,[j+1],i] = C_lopinf_dt@X_hat_rom_lopinf_dt[:,[j+1],i]

In [ ]:
# Uncenter FOM state and output data with initial conditions for each episode
for i in range(n_train+n_test):
    Y_hat_rom_lopinf_dt[:,:,i] = Y_hat_rom_lopinf_dt[:,:,i] + Y0[:,i].reshape(m,1)

In [ ]:
# Compute centered state responses

# Allocate space and convert rom responses to dask arrays
basis_dmdc = da.from_array(basis_dmdc,chunks = (4096,n_ROM))
basis_lopinf = da.from_array(basis_lopinf,chunks = (4096,n_ROM))
X_hat_rom_dmdc = da.from_array(X_hat_rom_dmdc,chunks = (4096,n_ROM,1))
X_hat_rom_lopinf_dt = da.from_array(X_hat_rom_lopinf_dt,chunks = (4096,n_ROM,1))
# Compute full state responses from roms and bases for each trial
X_hat_fom_dmdc = da.zeros((n,n_timesteps,n_train+n_test), chunks=(4096, n_timesteps,1))
X_hat_fom_lopinf = da.zeros((n,n_timesteps,n_train+n_test), chunks=(4096, n_timesteps,1))
for i in range(n_train + n_test):
    X_hat_fom_dmdc[:,:,i] = basis_dmdc@X_hat_rom_dmdc[:,:,i].reshape(-1,n_timesteps)
    X_hat_fom_lopinf_dt[:,:,i] = basis_lopinf@X_hat_rom_lopinf_dt[:,:,i].reshape(-1,n_timesteps)
    print(i)


### Uncenter Full Order Model

In [ ]:
# # Uncenter FOM state and output data with initial conditions for each episode
# for i in range(n_train+n_test):
#     X_fom[:,:,i] = X_fom[:,:,i] + X0[:,i].reshape(n,1)
#     Y_fom[:,:,i] = Y_fom[:,:,i] + Y0[:,i].reshape(m,1)
    

# Read in all again data to get uncentered versions
data = h5py.File(filepath+ "trainingSet.hdf5", 'r')
print("gothere")
# X_fom = da.from_array(data["stateData"][:,:,n_train+n_test], chunks=(4096, 4096,1))
U_fom = da.from_array(data["inputData"][:,:,n_train+n_test], chunks=(4096, 4096,1))
Y_fom = da.from_array(data["reducedCenterlineData"][:,:,n_train+n_test], chunks=(4096, 4096,1))

## Response Visualizations

In [ ]:
t = np.arange(0,dt*n_timesteps,dt)

### First Output Response

#### Training Set 1

In [ ]:
# Plot first output from first training episode for all modelling methods
plt.figure()
plt.plot(t,Y_fom[0,:,0],'k',label='FOM',linewidth=6)

plt.plot(t,Y_hat_rom_era[0,:,0],'r',label='ERA',linewidth=2)
# plt.plot(t,Y_hat_rom_dmdc[0,:,0],'b',label='DMDc',linewidth=5)
#plt.plot(t,Y_hat_rom_opInf[0,:,0],'g',label='OpInf',linewidth=5)
plt.plot(t,Y_hat_rom_lopinf[0,:,0],'m',label='LOpInf',linewidth=2)
plt.plot(t,Y_hat_rom_lopinf_dt[0,:,0],'c',label='LOpInf DT',linewidth=2)

legend = plt.legend(loc='upper right', shadow=False, fontsize='medium')
plt.xlabel('Time (s)')
plt.ylabel('Output')
title = 'First Output from First Training Episode'
plt.title(title)

#### Test Set 1

In [ ]:
# Plot first output from first testing episode for all modelling methods
outputIdx = 3
plt.figure()
plt.plot(t,Y_fom[outputIdx,:,n_train],'k',label='FOM',linewidth=5)
plt.plot(t,Y_hat_rom_era[outputIdx,:,n_train],'r',label='ERA',linewidth=2)
# plt.plot(t,Y_hat_rom_dmdc[outputIdx,:,n_train],'b',label='DMDc',linewidth=5)
# plt.plot(t,Y_hat_rom_opInf[outputIdx,:,n_train],'g',label='OpInf',linewidth=5)
plt.plot(t,Y_hat_rom_lopinf[outputIdx,:,n_train],'m',label='LOpInf',linewidth=2)
plt.plot(t,Y_hat_rom_lopinf_dt[outputIdx,:,n_train],'c',label='LOpInf DT',linewidth=2)
legend = plt.legend(loc='upper right', shadow=False, fontsize='medium')
plt.xlabel('Time (s)')
plt.ylabel('Output')
title = 'First Output from First Testing Episode'
plt.title(title)


### State Response

#### First Training Episode

In [ ]:
# Plot first state from first training episode for all modelling methods except ERA
plt.figure()
stateIdx = 0
plt.plot(t,X_fom[stateIdx,:,0] + x0[stateIdx],'k',label='FOM',linewidth=3)
# plt.plot(t,X_hat_fom_dmdc[stateIdx,:,0],'b',label='DMDc',linewidth=5)
plt.plot(t,X_hat_rom_lopinf[stateIdx,:,0],'m',label='LOpInf',linewidth=3)
plt.plot(t,X_hat_fom_lopinf_dt[stateIdx,:,0] +x0[stateIdx],'g',label='LOpInf DT',linewidth=2)
legend = plt.legend(loc='upper right', shadow=False, fontsize='medium')
plt.xlabel('Time (s)')
plt.ylabel('State')
title = 'First State from First Training Episode'
plt.title(title)

#### First Test Episode

In [ ]:
# Plot first state from first testing episode for all modelling methods except ERA
plt.figure()
stateIdx = 1000
plt.plot(t,X_fom[stateIdx,:,n_train]+ x0[stateIdx],'k',label='FOM',linewidth=3)
# plt.plot(t,X_hat_fom_dmdc[stateIdx,:,n_train],'b',label='DMDc',linewidth=5)
#plt.plot(t,X_hat_fom_opInf[stateIdx,:,n_train],'g',label='OpInf',linewidth=5)
plt.plot(t,X_hat_rom_lopinf[stateIdx,:,n_train],'m',label='LOpInf',linewidth=3)
plt.plot(t,X_hat_fom_lopinf_dt[stateIdx,:,n_train]+ x0[stateIdx],'g',label='LOpInf DT',linewidth=2)
legend = plt.legend(loc='upper right', shadow=False, fontsize='medium')
plt.xlabel('Time (s)')
plt.ylabel('State')
title = 'First State from First Testing Episode'
plt.title(title)

### Centerline Videos

In [ ]:
# Setup save path for videos
video_path = config["currentDirectory"] +"data/videos/"
videoName_test = 'test_episode_3segment.mp4'
videoName_train = 'train_episode_3segment.mp4'
# Number of frames and playspeed for video
numFrames = n_timesteps
fps = 30
duration = numFrames/fps
# Axis limits for video
xlim_max = 1150
xlim_min =  -300
zlim_max = -500
zlim_min = 500
ulim_max = 0.2
ulim_min = -0.2
# # Parameters for centerline reduction 
# n_seg = 10 # number of discrete points to use for centerline
# N_local = 20 # number of local points to use when averaging around each discrete point



#### Training Set 1

Generate animation of centerline motion using output data from all modelling methods for the first training set


In [ ]:
# # Set up figure
# fig, ax = plt.subplots()
# # Define animation callback
# def animate(t):
#     # Bring in global variables for output data of fom and each method
#     global Y_fom, Y_hat_rom_era, Y_hat_rom_dmdc, Y_hat_rom_opInf, Y_hat_rom_lopinf
#     # Get frame index
#     i = int(round(t*fps))
#     # Reshape centerline data for each method for the given timestep of the first training episode
#     centerline_fom = Y_fom[:,i,0].reshape(-1,2)
#     centerline_era = Y_hat_rom_era[:,i,0].reshape(-1,2)
#     centerline_dmdc = Y_hat_rom_dmdc[:,i,0].reshape(-1,2)
#     #centerline_opInf = Y_hat_rom_opInf[:,i,0].reshape(-1,2)
#     centerline_lopinf = Y_hat_rom_lopinf[:,i,0].reshape(-1,2)
#     # Compute reduced centerlines for each method
#     centerline_fom_red = centerline_fom
#     centerline_era_red = centerline_era
#     centerline_dmdc_red = centerline_dmdc
#     centerline_lopinf_red = centerline_lopinf
#     #centerline_opInf_red = reduceCenterline(n_seg,centerline_opInf,N_local)
#     # centerline_lopinf_red = reduceCenterline(n_seg,centerline_lopinf,N_local)
#     # Plot reduced centerlines
#     ax.clear()
#     ax.plot(centerline_fom_red[:,1],centerline_fom_red[:,0],'k',label='FOM',linewidth=5)
#     ax.plot(centerline_era_red[:,1],centerline_era_red[:,0],'r',label='ERA',linewidth=5)
#     ax.plot(centerline_dmdc_red[:,1],centerline_dmdc_red[:,0],'b',label='DMDc',linewidth=5)
#     #ax.plot(centerline_opInf_red[:,0],centerline_opInf_red[:,1],'g',label='OpInf',linewidth=5)
#     ax.plot(centerline_lopinf_red[:,1],centerline_lopinf_red[:,0],'m',label='LOpInf',linewidth=5)
#     ax.set_xlim(xlim_min,xlim_max)
#     ax.set_ylim(zlim_min,zlim_max)
#     ax.set_xlabel('x (mm)')
#     ax.set_ylabel('z (mm)')
#     ax.set_title('Centerline at timestep '+str(i))
#     legend = plt.legend(loc='upper right', shadow=False, fontsize='medium')
#     return mplfig_to_npimage(fig)

# animation = VideoClip(animate, duration=duration)
# animation.write_videofile(video_path+videoName_train, fps=fps)



#### Test Set 1

In [ ]:
# # Set up figure
# fig, ax = plt.subplots()
# # Define animation callback 
# def animate(t):
#     # Bring in global variables for output data of fom and each method
#     global Y_fom, Y_hat_rom_era, Y_hat_rom_dmdc, Y_hat_rom_opInf, Y_hat_rom_lopinf
#     # Get frame index
#     i = int(round(t*fps))
#     # Reshape centerline data for each method for the given timestep of the first testing episode
#     centerline_fom = Y_fom[:,i,n_train].reshape(-1,2)
#     centerline_era = Y_hat_rom_era[:,i,n_train].reshape(-1,2)
#     centerline_dmdc = Y_hat_rom_dmdc[:,i,n_train].reshape(-1,2)
#     #centerline_opInf = Y_hat_rom_opInf[:,i,0].reshape(-1,2)
#     centerline_lopinf = Y_hat_rom_lopinf[:,i,n_train].reshape(-1,2)
#     # Compute reduced centerlines for each method
#     centerline_fom_red = centerline_fom
#     centerline_era_red = centerline_era
#     centerline_dmdc_red = centerline_dmdc
#     centerline_lopinf_red = centerline_lopinf
#     #centerline_opInf_red = reduceCenterline(n_seg,centerline_opInf,N_local)
#     # centerline_lopinf_red = reduceCenterline(n_seg,centerline_lopinf,N_local)
#     # Plot reduced centerlines
#     ax.clear()
#     ax.plot(centerline_fom_red[:,1],centerline_fom_red[:,0],'k',label='FOM',linewidth=5)
#     ax.plot(centerline_era_red[:,1],centerline_era_red[:,0],'r',label='ERA',linewidth=5)
#     ax.plot(centerline_dmdc_red[:,1],centerline_dmdc_red[:,0],'b',label='DMDc',linewidth=5)
#     #ax.plot(centerline_opInf_red[:,0],centerline_opInf_red[:,1],'g',label='OpInf',linewidth=5)
#     ax.plot(centerline_lopinf_red[:,1],centerline_lopinf_red[:,0],'m',label='LOpInf',linewidth=5)
#     ax.set_xlim(xlim_min,xlim_max)
#     ax.set_ylim(zlim_min,zlim_max)
#     ax.set_xlabel('x (mm)')
#     ax.set_ylabel('z (mm)')
#     ax.set_title('Centerline at timestep '+str(i))
#     legend = plt.legend(loc='upper right', shadow=False, fontsize='medium')
#     return mplfig_to_npimage(fig)

# animation = VideoClip(animate, duration=duration)
# animation.write_videofile(video_path+videoName_test, fps=fps)



## Relative Error Computations

### Relative Output Errors

#### Training Episodes

In [ ]:
# # Compute relative error in frobenius norm for all training episodes
# rel_err_era_train = np.zeros((n_train,1))
# rel_err_dmdc_train = np.zeros((n_train,1))
# rel_err_lopinf_train = np.zeros((n_train,1))
# for i in range(n_train):
#     rel_err_era_train[i] = np.linalg.norm(Y_fom[:,:,i]-Y_hat_rom_era[:,:,i])/np.linalg.norm(Y_fom[:,:,i])
#     rel_err_dmdc_train[i] = np.linalg.norm(Y_fom[:,:,i]-Y_hat_rom_dmdc[:,:,i])/np.linalg.norm(Y_fom[:,:,i])
#     rel_err_lopinf_train[i] = np.linalg.norm(Y_fom[:,:,i]-Y_hat_rom_lopinf[:,:,i])/np.linalg.norm(Y_fom[:,:,i])

#### Test Episodes


In [ ]:
# # Compute relative error in frobenius norm for all testing episodes
# rel_err_era_test = np.zeros((n_test,1))
# rel_err_dmdc_test = np.zeros((n_test,1))
# rel_err_opInf_test = np.zeros((n_test,1))
# rel_err_lopinf_test = np.zeros((n_test,1))
# for i in range(n_test):
#     rel_err_era_test[i] = np.linalg.norm(Y_fom[:,:,i+n_train]-Y_hat_rom_era[:,:,i+n_train])/np.linalg.norm(Y_fom[:,:,i+n_train])
#     rel_err_dmdc_test[i] = np.linalg.norm(Y_fom[:,:,i+n_train]-Y_hat_rom_dmdc[:,:,i+n_train])/np.linalg.norm(Y_fom[:,:,i+n_train])
#     rel_err_lopinf_test[i] = np.linalg.norm(Y_fom[:,:,i+n_train]-Y_hat_rom_lopinf[:,:,i+n_train])/np.linalg.norm(Y_fom[:,:,i+n_train])

In [ ]:
# # Make bar plot with error bars for training and testing episodes for each method
# fig, ax = plt.subplots()
# ax.bar(np.arange(0,8,2)-0.45,[np.mean(rel_err_era_train),np.mean(rel_err_dmdc_train),np.mean(rel_err_opInf_train),np.mean(rel_err_lopinf_train)],yerr=[np.std(rel_err_era_train),np.std(rel_err_dmdc_train),np.std(rel_err_opInf_train),np.std(rel_err_lopinf_train)],color=['r','b','g','m'],align='center',alpha=0.5,ecolor='black',capsize=10)
# ax.bar(np.arange(0,8,2)+0.45,[np.mean(rel_err_era_test),np.mean(rel_err_dmdc_test),np.mean(rel_err_opInf_test),np.mean(rel_err_lopinf_test)],yerr=[np.std(rel_err_era_test),np.std(rel_err_dmdc_test),np.std(rel_err_opInf_test),np.std(rel_err_lopinf_test)],color=['r','b','g','m'],align='center',alpha=0.5,ecolor='black',capsize=10,hatch='//')
# ax.set_ylabel('Relative error')
# ax.set_xticks(np.arange(0,8,2))
# ax.set_xticklabels(['ERA','DMDc','OpInf','LOpInf'])
# ax.set_title('Relative output error for training and testing episodes')
# ax.yaxis.grid(True)
# ax.legend(['Training','Testing'])
# # plt.tight_layout()


### Relative State Errors

In [ ]:
# # Compute relative error in frobenius norm for all training episodes of full order states for each method except ERA
# rel_err_dmdc_train_full = np.zeros((n_train,1))
# rel_err_opInf_train_full = np.zeros((n_train,1))
# rel_err_lopinf_train_full = np.zeros((n_train,1))
# for i in range(n_train):
#     rel_err_dmdc_train_full[i] = np.linalg.norm(X_fom[:,:,i]-X_hat_fom_dmdc[:,:,i])/np.linalg.norm(X_fom[:,:,i])
#     rel_err_opInf_train_full[i] = np.linalg.norm(X_fom[:,:,i]-X_hat_fom_opInf[:,:,i])/np.linalg.norm(X_fom[:,:,i])
#     # rel_err_lopinf_train_full[i] = np.linalg.norm(X_fom[:,:,i]-X_hat_rom_lopinf[:,:,i])/np.linalg.norm(X_fom[:,:,i])
    


In [ ]:
# # Compute relative error in frobenius norm for all testing episodes of full order states for each method except ERA
# rel_err_dmdc_test_full = np.zeros((n_test,1))
# rel_err_opInf_test_full = np.zeros((n_test,1))
# rel_err_lopinf_test_full = np.zeros((n_test,1))
# for i in range(n_test):
#     rel_err_dmdc_test_full[i] = np.linalg.norm(X_fom[:,:,i+n_train]-X_hat_fom_dmdc[:,:,i+n_train])/np.linalg.norm(X_fom[:,:,i+n_train])
#     rel_err_opInf_test_full[i] = np.linalg.norm(X_fom[:,:,i+n_train]-X_hat_fom_opInf[:,:,i+n_train])/np.linalg.norm(X_fom[:,:,i+n_train])
#     # rel_err_lopinf_test_full[i] = np.linalg.norm(X_fom[:,:,i+n_train]-X_hat_rom_lopinf[:,:,i+n_train])/np.linalg.norm(X_fom[:,:,i+n_train])



In [ ]:
# # Make bar plot with error bars for training and testing episodes for each method
# fig, ax = plt.subplots()
# ax.bar(np.arange(0,6,2)-0.45,[np.mean(rel_err_dmdc_train_full),np.mean(rel_err_opInf_train_full),np.mean(rel_err_lopinf_train_full)],yerr=[np.std(rel_err_dmdc_train_full),np.std(rel_err_opInf_train_full),np.std(rel_err_lopinf_train_full)],color=['b','g','m'],align='center',alpha=0.5,ecolor='black',capsize=10)
# ax.bar(np.arange(0,6,2)+0.45,[np.mean(rel_err_dmdc_test_full),np.mean(rel_err_opInf_test_full),np.mean(rel_err_lopinf_test_full)],yerr=[np.std(rel_err_dmdc_test_full),np.std(rel_err_opInf_test_full),np.std(rel_err_lopinf_test_full)],color=['b','g','m'],align='center',alpha=0.5,ecolor='black',capsize=10,hatch='//')
# ax.set_ylabel('Relative error')
# ax.set_xticks(np.arange(0,6,2))
# ax.set_xticklabels(['DMDc','OpInf','LOpInf'])
# ax.set_title('Relative full state error for training and testing episodes')
# ax.yaxis.grid(True)
# ax.legend(['Training','Testing'])
# # plt.tight_layout()

